# Notebook 12.1: Hi-C scaffolding

### Learning objectives:
By the end of this notebook you should:
+ Understand the concepts behind Hi-C.
+ Be familiar with code and programs for scaffolding with Hi-C reads. 

### Assigned reading:

+ Lightfoot, D. J., D. E. Jarvis, T. Ramaraj, R. Lee, E. N. Jellen, and P. J. Maughan. 2017. “Single-Molecule Sequencing and Hi-C-Based Proximity-Guided Assembly of Amaranth (Amaranthus Hypochondriacus) Chromosomes Provide Insights into Genome Evolution.” BMC Biology 15 (August): 74. https://doi.org/10.1186/s12915-017-0412-4.


+ Burton, Joshua N., Andrew Adey, Rupali P. Patwardhan, Ruolan Qiu, Jacob O. Kitzman, and Jay Shendure. 2013. “Chromosome-Scale Scaffolding of de Novo Genome Assemblies Based on Chromatin Interactions.” Nature Biotechnology 31 (12): 1119–25. https://doi.org/10.1038/nbt.2727.


+ [See video on library preparation for Hi-C](https://www.jove.com/video/1869/hi-c-a-method-to-study-the-three-dimensional-architecture-of-genomes) 
    

### Required software
We can install the required software for Hi-C scaffolding using the conda package manager. The code is included for reference but commented out because we have already install all necessary software for you. 

In [1]:
# conda install numpy pandas
# conda install toyplot -c eaton-lab
# conda install sratools -c bioconda
# conda install bwa -c bioconda
# conda install hicexplorer -c bioconda -c conda-forge

In [2]:
import numpy as np
import pandas as pd
import toyplot

### Chromosome conformation capture (3C)

Chromosome conformation capture (3C) methods were first developed for studying the conformation (structure and organization) of the genome within cells. Does the genome always fold in predictable ways, do certain chromosomes always exist close together or far apart in the cell? These questions are hard to answer even with microscopy. The idea behind Hi-C was to develop a method based on sequence information that could tell us about the spatial organization of the genome. It decodes the spatial organization of the genome from *interactions* among parts of the genome that occur in a way that correlates with the spatial distance between them. The 3C method was developed as a way to label individual loci and measure the distance between them in a living cell. The **Hi-C** method is a new advance on this original method, extending it to a rapid genome-wide process. 

### Hi-C  chromosome interactions
Start by watching [this video](https://www.jove.com/video/1869/hi-c-a-method-to-study-the-three-dimensional-architecture-of-genomes) which explains the concept behind Hi-C and how to prepare a Hi-C library for sequencing. As you watch pay attention as they describe some of the molecular techniques involved. We will return to these over the next few weeks in greater detail. Two important steps are *restriction digestion* and *biotin bait capture*. Let's explore the first of these through some Python coding.

### Generate restriction digested genome fragments
Restriction enzymes are enzymes that recognize and bind to DNA at certain sites where a specific recognition pattern is found. These enzymes occur naturally in bacteria where they serve a useful function for cutting up viral genomes as a defense mechanism. They've also turned out be a very useful tool for molecular biology. Hundreds of restriction enzymes have now been discovered which recognize different enzyme recognition sites. These sites are often palindromes from 3-10 bp in length. 

When a genome is digested with a restriction enzyme the genome is broken into smaller fragments. Each fragment will begin and end with a characteristic *overhang* of the restriction enzyme. For the restriction enzyme `HindIII`, the recognition site is `AAGCTT`, and the cut occurs between the two A's in the 5' direction (`A^AGCTT`) such that it leaves one `A` at the end a fragment, and `AGCTT` at the beginning of the next fragment. Let's see what this looks like:

In [3]:
def random_sequence(length):
    "return a random sequence of DNA"
    return "".join(np.random.choice(list("ACGT"), size=length))

In [4]:
def restriction_digest(sequence, recognition, cut):
    """
    restriction digest a genome sequence at the given (recognition) site and
    split the site at the given position (cut) to leave overhangs. 
    """
    # cut sequence at every occurence of recognition site
    fragments = sequence.split(recognition)
    
    # add overhang that results from sequence splitting within the recognition site
    fragments = [recognition[cut:] + i + recognition[:cut] for i in fragments]
    return fragments

### Cut up a genome with HindIII

In [5]:
# generate a 5Mb genome
seq = random_sequence(5000000)

# digest the genome at every HindIII site
fragments = restriction_digest(seq, "AAGCTT", 1)

In [6]:
# print headers
print("Restriction recognition site: A^AGCTT")
print("Expected: [overhang-AGCTT][sequence][overhang-A]")

# check the beginning and end of the first 10 fragments
for i in range(10):
    print(fragments[i][:5], fragments[i][5:10], '...', fragments[i][-1:])

Restriction recognition site: A^AGCTT
Expected: [overhang-AGCTT][sequence][overhang-A]
AGCTT TCAGA ... A
AGCTT GTCAA ... A
AGCTT TTGCG ... A
AGCTT ACGTT ... A
AGCTT CGGGT ... A
AGCTT GTGAA ... A
AGCTT ACGTC ... A
AGCTT TCTCG ... A
AGCTT GCTCA ... A
AGCTT GAAGC ... A


### Look at the fragmented genome size distribution

In [7]:
# get fragment lengths binned 
flens = np.histogram([len(i) for i in fragments], bins=50)

In [8]:
# plot distribution of fragment lengths
toyplot.bars(
    flens,
    width=400,
    height=300,
    xlabel="fragment size", 
    ylabel="number of fragments",
);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t258f9d6b760c48fdb8b44b6521d481e2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 10000 20000 30000 fragment size 0 50 100 150 number of fragments

<div class="alert alert-success">
    <b>[1] Action:</b> 
    Look up the restriction recognition site for the restriction enzyme `PstI` (e.g., search on google). Modify and reuse the code from above to digest the same genome using this enzyme and plot the distribution of fragment lengths below. Copy and modify the print statement from above also to ensure that the proper overhangs are found on the ends of your digested fragments. 
</div>

In [14]:
# generate a 5Mb genome
seq = random_sequence(5000000)

# digest the genome at every HindIII site
fragments = restriction_digest(seq, "CTGCAG", 5)

# print headers
print("Restriction recognition site: CTGCA^G")
print("Expected: [overhang-G][sequence][overhang-CTGCA]")

# check the beginning and end of the first 10 fragments
for i in range(10):
    print(fragments[i][:1], fragments[i][5:10], '...', fragments[i][-5:])

# get fragment lengths binned 
flens = np.histogram([len(i) for i in fragments], bins=50)

# plot distribution of fragment lengths
toyplot.bars(
    flens,
    width=400,
    height=300,
    xlabel="fragment size", 
    ylabel="number of fragments",
);

Restriction recognition site: CTGCA^G
Expected: [overhang-G][sequence][overhang-CTGCA]
G CGAGC ... CTGCA
G CAATG ... CTGCA
G CTCCA ... CTGCA
G TTAGT ... CTGCA
G AAACT ... CTGCA
G ATGTC ... CTGCA
G GTATG ... CTGCA
G AGGCA ... CTGCA
G ACCAA ... CTGCA
G TAGTT ... CTGCA


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t3c3a5bdeea684e0ea34326399c1e82fd" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 10000 20000 30000 fragment size 0 50 100 150 number of fragments

### Fragments in close proximity will rejoin with higher probability

We now have a list (`fragments`) that contains the restriction digested fragments of the genome ordered in the same order as the original genome. If we wanted to know the original genome sequence from these fragments we could just stitch them together. However, when we digest a genome in real life we will not receive the fragments in the correct order but instead their order will be random. Our task then will be to try to figure out the order of these sequences. This is the type of data that Hi-C provides in the form of a contact map, or a matrix file, that records how often reads from two different parts of the genome interact.

In Hi-C this is done by *cross-linking* the genome before the restriction digestion. This makes the DNA bind to the histone proteins near to it so that when it is digested the fragments remain close to their original location. After digestion the overhanging ends are repaired and marked with `biotin`, a metal *that allows us to pull this bit of DNA out of solution later using magnets!*. The digested and repaired ends from different fragments can then ligate to each other. Most of the time we expect the closest fragment to recombine with one will be the same original fragment it was broken from, but some of the time the fragments will rejoin with other fragments that are from the same chromosome, or even sometimes from other chromosomes that are located close spatially. 

The resulting *chimeric* sequences (composed of two regions of the genome that are now joined together) can be pulled out of the solution and sequenced using paired-end Illumina sequencing. When we map the pairs to a reference genome the number of times that pairs map to two different regions corresponds to the spatial proximity of those regions.  


![https://github.com/genomics-course/12-scaffolding/raw/master/data/Hi-C.1.png](https://github.com/genomics-course/12-scaffolding/raw/master/data/Hi-C.1.png)

### Tools for Hi-C data analysis

[ALLHiC](https://github.com/tangerzhang/ALLHiC/wiki), [HiCExplorer](https://hicexplorer.readthedocs.io/en/latest/index.html) and [Juicer](https://github.com/aidenlab/juicer/wiki) are just a few of many tools that are now available for analyzing Hi-C data and for visualizing the results. I found that AllHic and HiCExplorers fairly straight forward to use whereas Juicer seems very complicated to install and run. These programs all follow a similar approach that includes the following:


1. Map Hi-C paired-end reads to an indexed draft reference genome. 
2. Filter out bad pairs (Hi-C can be messy).
3. Infer a contact matrix based on mapped positions of the pairs.
4. Correct the contact matrix by normalizing read counts (repetitions).
5. Visualize genome structure based on contact matrix.
6. Scaffold genome based on contact matrix.


The AllHiC method:

![AllHiC](https://camo.githubusercontent.com/768f74b34e48c46f709c98399be411df19ba2fdc/68747470733a2f2f7777772e64726f70626f782e636f6d2f732f617369616577347931343261636d632f414c4c4869432d4f766572766965772e706e673f7261773d31)

### Amaranthus example
Your reading by Lightfoot et al. describes the improved scaffolding of the *Amaranthus* genome using long-reads, Hi-C, and optical mapping. Let's focus on the application of Hi-C data to this problem here. 

**Because Hi-C scaffolding is very computationally intensive we will not actually run the code here, but simply follow along and look at the results. Leave the code commented out.**

First, we can download the draft v.1.0 *Amaranthus* genome from NCBI and decompress it.

In [212]:
%%bash

# wget -q -O Amaranthus_v1.fna.gz https://ftp.ncbi.nlm.nih.gov/genomes/genbank/plant/Amaranthus_hypochondriacus/all_assembly_versions/GCA_000753965.1_AHP_1.0/GCA_000753965.1_AHP_1.0_genomic.fna.gz
# gunzip Amaranthus_v1.fna.gz


Then we'll index the reference sequence so it's ready for read mapping.

In [211]:
%%bash

# bwa index -a bwtsw Amaranthus_v1.fna
# samtools faidx Amaranthus_v1.fna

Now download the paired-end Illumina data from the Lightfoot et al. article using the `fastq-dump` tool from the sra-tools. 

In [200]:
%%bash

# fastq-dump SRR5345531 --split-files

### Map the paired reads to the reference

Here we use `bwa` a read mapping tool we have seen before. We're using a different algorithm than usual (`bwa aln` instead of `bwa mem`) as this is more appropriate for mapping paired-end reads that might map very far from each other (which is our expectation). The `aln` call aligns the forward and reverse reads to the genome separately. Then the `sampe` call generates a SAM output of how the pairs align together to the genome. This contains the information we are interested in, the number of times paired reads mapped to each region of the genome.

In [224]:
%%bash

# bwa aln -t 20 Amaranthus_v1.fna SRR5345531_1.fastq > SRR5345531_1.sai  
# bwa aln -t 20 Amaranthus_v1.fna SRR5345531_2.fastq > SRR5345531_2.sai  
# bwa sampe \
#     Amaranthus_v1.fna \
#     SRR5345531_1.sai SRR5345531_2.sai \
#     SRR5345531_1.fastq SRR5345531_2.fastq \
#     > SRR5345531.bwa_aln.sam 

### Clean the mapping results
In this section we call two tools from the `ALLHiC` package to clean up the mapping information. These check that all of the reads are paired properly near a restriction site as would be expected of proper HiC data. Then we use samtools to convert the cleaned SAM file to binary format (BAM). 

In [225]:
%%bash

# PreprocessSAMs.pl \
#     SRR5345531.bwa_aln.bam \
#     Amaranthus_v1.fna \
#     HINDIII

# filterBAM_forHiC.pl \
#     SRR5345531.bwa_aln.REduced.paired_only.bam \
#     SRR5345531.clean.sam
    
# samtools view -bt \
#     Amaranthus_v1.fna.fai \
#     SRR5345531.clean.sam > SRR5345531.clean.bam

### Build the contact matrix
One of the resulting files, `Allele.ctg.table`, contains the table of contacts between different restriction digested fragments of the genome. We can look at a few lines of this table with pandas. We can see that it shows us the top 3 hits for a marker ("tigXXXXXXX") at each restriction digested region and where it maps to. For example, the marker at position 11180 (labeled tig00071149) matched to tig00012288, tig00067986, and tig00011375. Because the positions are binned these numbers do not match to exact positions of genome, but rather windows. 


The next steps in the ALLHiC pipeline involves correcting this table of contact positions using the mapping information again to remove spurious mappings. 

In [242]:
pd.read_csv(
    "https://raw.githubusercontent.com/genomics-course/12-scaffolding/master/data/Allele.ctg.table",
    sep="\t",
    names=["chrom", "pos", "marker", "hit1", "hit2", "hit3"],
    header=None,
).head(10)

,chrom,pos,marker,hit1,hit2,hit3
0,Chr01,1951,tig00067987,tig00067986,tig00082238,NaN
1,Chr01,11180,tig00071149,tig00012288,tig00067986,tig00011375
2,Chr01,22391,tig00063432,tig00063431,tig00071150,tig00071150
3,Chr01,53781,tig00037808,tig00053424,tig00037808,NaN
4,Chr01,62892,tig00044711,tig00044711,tig00044711,NaN
5,Chr01,79159,tig00044711,tig00075146,tig00075146,NaN
6,Chr01,112766,tig00075146,tig00044710,tig00044711,tig00044711
7,Chr01,155315,tig00044710,tig00044708,tig00044709,tig00048805
8,Chr01,162161,tig00048805,tig00044708,tig00002498,tig00044710
9,Chr01,185033,tig00065599,tig00058222,tig00065599,tig00058222


In [243]:
%%bash

# ### Prune
# ALLHiC_prune \
#     -i Allele.ctg.table \
#     -b SRR5345531.clean.bam \
#     -r Amaranthus_v1.fna

# ### Partition into k groups
# ALLHiC_partition \
#     -r Amaranthus_v1.fna \
#     -e AAGCTT \
#     -k 16

# ### Rescue
# ALLHiC_rescue \
#     -r Amaranthus_v1.fna \
#     -b SRR5345531.clean.bam \
#     -c prunning.clusters.txt \
#     -i prunning.counts_AAGCTT.txt

# ### Optimize
# allhic extract \
#     SRR5345531.clean.bam \
#     Amaranthus_v1.fna \
#     -RE AAGCTT

# for k in {1..16}; 
#   do allhic optimize group${k}.txt SRR5345531.clean.clm;
# done

### Scaffolding
Finally once the contact map has been refined we can call `build` from ALLHiC to get a newly scaffolded genome file. The steps involved above are very computationally intensive so this analysis is still running for me. We will analyze the results in class together on Wed. 

In [244]:
%%bash

# ### Build 
# ALLHiC_build Amaranthus_v1.fna

In [245]:
%%bash

#ALLHiC_plot SRR5345531.clean.bam groups.agp chrn.list 500k pdf

### We'll examine the final results of this in class. 

That's it for now, move on to the next notebook.